<a href="https://colab.research.google.com/github/lisya/ML_in_business/blob/lesson06/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('/content/transfusion.data')
data.head(5)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


**У нас имеется датасет с данными доноров крови. Нобходимо определить сдавал ли донор кровь в марте 2007 или нет**

In [12]:
data.iloc[:, -1].value_counts()

0    570
1    178
Name: whether he/she donated blood in March 2007, dtype: int64

**Разобъем наш датасет на трейн и тест и обучим RandomForest**

In [36]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.5, random_state=7,stratify=y_data )

In [37]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [38]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 43.79%
roc: 63.24%
recall: 41.57%
precision: 46.25%


**PU learning**

In [44]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 45/178 as positives and unlabeling the rest


In [45]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    703
 1     45
Name: class_test, dtype: int64


In [46]:
mod_data.head(10)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007,class_test
0,2,50,12500,98,1,-1
1,0,13,3250,28,1,-1
2,1,16,4000,35,1,-1
3,2,20,5000,45,1,-1
4,1,24,6000,77,0,-1
5,4,4,1000,4,0,-1
6,2,7,1750,14,1,-1
7,1,12,3000,35,0,-1
8,2,9,2250,22,1,-1
9,5,46,11500,98,1,-1


In [47]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

**random negative sampling**

In [48]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(45, 6) (45, 6)


In [49]:
model = RandomForestClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 40.33%
roc: 64.26%
recall: 77.78%
precision: 27.22%


**Посмотрим как меняются метрики при изменении количества позитивов.**

In [54]:
p_list = np.arange(0.1, 1.0, 0.1)

In [61]:
for p in p_list:
  mod_data = data.copy()
  #get the indices of the positives samples
  pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
  #shuffle them
  np.random.shuffle(pos_ind)
  # leave just 25% of the positives marked
  pos_sample_len = int(np.ceil(p * len(pos_ind)))
  print(f'p = {round(p*100,1)}%')
  print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
  pos_sample = pos_ind[:pos_sample_len]

  mod_data['class_test'] = -1
  mod_data.loc[pos_sample,'class_test'] = 1

  x_data = mod_data.iloc[:,:-2].values # just the X 
  y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
  y_positive = mod_data.iloc[:,-2].values # original class

  mod_data = mod_data.sample(frac=1)
  neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
  sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
  pos_sample = mod_data[mod_data['class_test']==1]
  sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)


  model = RandomForestClassifier()

  model.fit(sample_train.iloc[:,:-2].values, 
            sample_train.iloc[:,-2].values)
  y_predict = model.predict(sample_test.iloc[:,:-2].values)
  evaluate_results(sample_test.iloc[:,-2].values, y_predict)
  print(f'\n')
  print(f'*'*50)
  print(f'\n')
  

p = 10.0%
Using 18/178 as positives and unlabeling the rest
Classification results:
f1: 41.68%
roc: 61.20%
recall: 85.90%
precision: 27.52%


**************************************************


p = 20.0%
Using 36/178 as positives and unlabeling the rest
Classification results:
f1: 39.48%
roc: 61.99%
recall: 68.15%
precision: 27.79%


**************************************************


p = 30.0%
Using 54/178 as positives and unlabeling the rest
Classification results:
f1: 35.56%
roc: 61.94%
recall: 77.27%
precision: 23.10%


**************************************************


p = 40.0%
Using 72/178 as positives and unlabeling the rest
Classification results:
f1: 36.36%
roc: 64.27%
recall: 71.43%
precision: 24.39%


**************************************************


p = 50.0%
Using 89/178 as positives and unlabeling the rest
Classification results:
f1: 31.02%
roc: 62.04%
recall: 70.00%
precision: 19.93%


**************************************************


p = 60.0%
Using 107/178 a